Стратегия объединяет BBANDS, RSI и определенные свечные паттерны для обнаружения точек разворота и определения оптимальных позиций входа.
- RSI - потдверждение тренда
- ATR - управления торговлей, используется для подсчета параметров стоп-лосса и тейк-профита <br>(класс BBRSIStrategy [backtest_strategy.ipynb](backtest_strategy.ipynb))

- apply_bb_rsi_strategy - основная функция сигналов и индикаторов
    - add_bb_rsi_strategy - добавляет индикаторы
    - add_bb_rsi_signal - определяет точки входа и выхода на базе индикаторов
- get_best_bb_rsi_strategy - основная функция поиска оптимальных параметров
- backest_bb_rsi_strategy - функция бэктестинга
- [walk_forward_optimization_by_index](./useful_functions.ipynb) - хелпер функция генерации wfo

In [1]:
import os
import pandas as pd
import warnings
import numpy as np
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from backtesting import Backtest
import seaborn as sns
import matplotlib.pyplot as plt
from datetime import datetime
import import_ipynb

from tqdm import tqdm
tqdm.pandas()
warnings.simplefilter('ignore')

In [2]:
# period=-2400
period=-(datetime.now() - datetime(2019, 1, 1)).days
time_interval='1d'
tickers = ['BTC-USD', 'ETH-USD'] #, 'SOL-USD', 'XRP-USD'
dir = "crypto_data"

In [38]:
%run crypto_data_loader.ipynb

yfinance version: 0.2.50
Start load crypto data with interval: 1d, from: 2019-01-01 16:26:24.892113


[*********************100%***********************]  4 of 4 completed


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 2178 entries, 2019-01-01 to 2024-12-17
Freq: D
Data columns (total 24 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   (BTC-USD, Open)       2178 non-null   float64
 1   (BTC-USD, High)       2178 non-null   float64
 2   (BTC-USD, Low)        2178 non-null   float64
 3   (BTC-USD, Close)      2178 non-null   float64
 4   (BTC-USD, Adj Close)  2178 non-null   float64
 5   (BTC-USD, Volume)     2178 non-null   int64  
 6   (SOL-USD, Open)       1713 non-null   float64
 7   (SOL-USD, High)       1713 non-null   float64
 8   (SOL-USD, Low)        1713 non-null   float64
 9   (SOL-USD, Close)      1713 non-null   float64
 10  (SOL-USD, Adj Close)  1713 non-null   float64
 11  (SOL-USD, Volume)     1713 non-null   float64
 12  (ETH-USD, Open)       2178 non-null   float64
 13  (ETH-USD, High)       2178 non-null   float64
 14  (ETH-USD, Low)        2178 non-null   float64


### Usefull functions

In [3]:
# %run file_loader.ipynb
# %run useful_functions.ipynb
# %run backtest_strategy.ipynb

from file_loader import loading_v3, get_data
from useful_functions import get_best_bb_rsi_strategy, walk_forward_optimization_by_index, apply_bb_rsi_strategy, add_pointpos_column
from backtest_strategy import BBRSIStrategy

# log.basicConfig(level=log.INFO)

### Loading data and Calculate Bollinger Bands and RSI and apply Signal

In [ ]:
report_dict = {}
symbols =['BTC-USD']
for name in symbols:
# for name in tickers:
    df = get_data(dir, name)
    df.reset_index(inplace=True)

    df_length = len(df)
    print(f"Start {name} backtesting, df: {df_length}")

    try:
        train_size = 200
        test_size = 33
        index_range = walk_forward_optimization_by_index(df_length, train_size, test_size)
        # print(pd.DataFrame(index_range))
        print(f"{name} wfo iterations: {len(index_range)}")

        params = {
            'bb_period_list': [15, 20, 30], # 13-30
            'bb_width_threshold_list': [0.002, 0.0015, 0.001],
            'atr_period_list': [7, 10, 14], # Long time frames: 20-50, Short: 2-10
            'rsi_period_list': [10, 10, 14], # Short-term intraday: 9-11, Medium-term: 14, Longer-term position: 20-30
            'cash': 10_000,
            # 'maximize': 'Profit Factor',
            'maximize': 'Return [%]',
            # 'maximize': 'Equity Final [$]', #!
            # 'maximize': 'Win Rate [%]',
            'max_tries': 300, # ???
            # 'stats_item': 'Profit Factor',
            'stats_item': 'Win Rate [%]',
            # 'method': 'skopt',
            'bb_std': 1.5, #1.8, 2
            'back_candles': 7, #8,9
            'ema_period': 30, # 20, 50, based on test_set_size param - ???
        }

        signals_df = pd.DataFrame()       
        for iter in tqdm(index_range):
            start_train = iter['train_indexes'][0]
            end_train = iter['train_indexes'][1]
            start_test = iter['test_indexes'][0]
            end_test = iter['test_indexes'][1]
            
            train_set = df.iloc[start_train:end_train].copy()
            test_set = df.iloc[start_test:end_test].copy()

            print(f"{name} : train_set: {train_set.iloc[0].Date} - {train_set.iloc[-1].Date}, test_set: {test_set.iloc[0].Date} - {test_set.iloc[-1].Date}")

            best_params, best_performance = get_best_bb_rsi_strategy(train_set, BBRSIStrategy, params, apply_bb_rsi_strategy)

            print(f"{name} : best_params: {best_params}, best_performance: {best_performance}")

            if best_performance > 0:
                combined_data = pd.concat([train_set, test_set]).reset_index(drop=True)
                combined_with_test_signal = apply_bb_rsi_strategy(combined_data.copy(), best_params)
                test_set_with_signal = combined_with_test_signal.iloc[-test_size:].copy()
                signals_df = pd.concat([signals_df, test_set_with_signal], ignore_index=True)

        if not signals_df.empty:
            report_dict[name] = signals_df
    except Exception as e:
        print(f"Error backtesting for {name}: {e}")

Start BTC-USD backtesting, df: 2178
BTC-USD wfo iterations: 60


  0%|          | 0/60 [00:00<?, ?it/s]

BTC-USD : train_set: 2019-01-01 00:00:00 - 2019-07-19 00:00:00, test_set: 2019-07-20 00:00:00 - 2019-08-21 00:00:00


Backtest.optimize:   0%|          | 0/9 [00:00<?, ?it/s]

Performance, current: 66.66666666666666, best_performance: -inf


...

Backtest.optimize:   0%|          | 0/9 [00:00<?, ?it/s]

Backtest.optimize:   0%|          | 0/9 [00:00<?, ?it/s]

Backtest.optimize:   0%|          | 0/9 [00:00<?, ?it/s]

Backtest.optimize:   0%|          | 0/9 [00:00<?, ?it/s]

Backtest.optimize:   0%|          | 0/9 [00:00<?, ?it/s]

### Run the backtest for the out-of-sample data using the optimal parameters.

In [16]:
report = dict()

for name, df in report_dict.items():
    bt = Backtest(df, BBRSIStrategy, cash=10_000, margin=1/10, commission=0.001, exclusive_orders=True)
    report[name] = bt.run()
    bt.plot(plot_equity=True,plot_drawdown=True,relative_equity=False,)

for name, stats in report.items():
    print(f"=== {name}:")
    print(stats)


=== BTC-USD:
Start                                     0.0
End                                     119.0
Duration                                119.0
Exposure Time [%]                   21.666667
Equity Final [$]                 10712.307598
Equity Peak [$]                  10712.307598
Return [%]                           7.123076
Buy & Hold Return [%]              -20.584658
Return (Ann.) [%]                         0.0
Volatility (Ann.) [%]                     NaN
Sharpe Ratio                              NaN
Sortino Ratio                             NaN
Calmar Ratio                              0.0
Max. Drawdown [%]                  -10.714245
Avg. Drawdown [%]                   -7.364487
Max. Drawdown Duration                   16.0
Avg. Drawdown Duration                   11.5
# Trades                                  1.0
Win Rate [%]                            100.0
Best Trade [%]                       8.573734
Worst Trade [%]                      8.573734
Avg. Trade [%]       

In [ ]:
report_dict['BTC-USD'].describe()

,Date,Open,High,Low,Close,bb_up,bb_mid,bb_low,bb_width,rsi,atr,Signal
count,2310,2310.000000,2310.000000,2310.000000,2310.000000,2107.000000,2107.000000,2107.000000,2107.000000,2212.000000,2212.000000,2310.000000
mean,2019-09-16 18:51:25.714285824,8307.040607,8492.406662,8117.741998,8315.217411,9409.088628,8433.211024,7457.333420,0.230780,52.280194,377.200137,0.043290
min,2019-01-01 00:00:00,3401.376465,3427.945557,3391.023682,3399.471680,3702.000543,3568.140169,3349.405293,0.060367,14.080537,63.400053,0.000000
25%,2019-06-21 00:00:00,7277.197754,7418.858887,7167.179199,7276.802734,8443.029132,7420.057520,6912.231121,0.154624,43.015584,270.783237,0.000000
50%,2019-09-17 00:00:00,8487.669922,8640.493164,8276.612305,8486.993164,9636.870780,8643.247282,7629.149413,0.209787,51.690641,332.314801,0.000000
75%,2019-12-10 00:00:00,9871.165039,10118.483643,9675.316406,9870.303711,10920.557152,10000.656087,8920.338399,0.275113,62.698759,429.956062,0.000000
max,2020-06-23 00:00:00,13017.125000,13796.489258,12233.261719,13016.231445,12713.528244,11213.690625,10008.061282,0.654091,90.781110,967.702870,2.000000
std,NaN,2093.928802,2153.498591,2014.899745,2085.889042,2145.242433,1884.669168,1741.911796,0.118654,14.426739,191.350021,0.269474


### Calculate signals count

In [17]:
sum([report_dict[key]['Signal'].value_counts() for key in report_dict], start=0)

Signal
0    117
2      3
Name: count, dtype: int64

In [14]:
# Checks the count of signals for each ticker
for df in report_dict.values():
    print(len(df[df.Signal != 0]))

6


## Position points plot


In [16]:
# Add position points
for df in report_dict.values():
    df = add_pointpos_column(df, "Signal", 1e-4)


Params: 0.0001


In [ ]:
st=0
for name, df in report_dict.items():
    dfpl = df[st:st+350]

    # Create a plot with 2 rows
    fig = make_subplots(rows=2, cols=1, column_titles=[name])

    # Add candlestick plot on the first row
    fig.add_trace(go.Candlestick(x=dfpl.index,
                                open=dfpl['Open'],
                                high=dfpl['High'],
                                low=dfpl['Low'],
                                close=dfpl['Close']),
                row=1, col=1)

    # Add Bollinger Bands, EMA lines on the same subplot
    fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['bb_low'],
                            line=dict(color='green', width=1),
                            name="BB_Low"),
                row=1, col=1)
    
    fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['bb_mid'],
                            line=dict(color='yellow', width=1),
                            name="BB_MID"),
                row=1, col=1)

    fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['bb_up'],
                            line=dict(color='green', width=1),
                            name="BB_UPPER"),
                row=1, col=1)

    # Add markers for trade entry points on the same subplot
    fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['pointpos'], mode="markers",
                            marker=dict(size=8, color="MediumPurple"),
                            name="entry"),
                row=1, col=1)

    # Add markers for trade entry points on the same subplot
    fig.add_trace(go.Scatter(x=dfpl.index, y=dfpl['rsi'], 
                            line=dict(color='brown', width=2),
                            name="RSI"),
                row=2, col=1)

    fig.update_layout(width=800, height=600, sliders=[])
    fig.show()
